The purpose of this notebook is to document process of preparation DinD image (Docker in Docker) with black-is-white feature.

We use source code from https://github.com/abbbe/moby repo, black_is_white branch.

Need access to registry to push. See common/Registry.ipynb.

## Install docker

https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository

*NB*: assume sudo does not ask for password

In [ ]:
sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg

In [5]:
sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
sudo chmod a+r /etc/apt/keyrings/docker.gpg

In [6]:
echo \
  "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
  "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

In [ ]:
sudo apt-get update

In [ ]:
sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin

In [10]:
sudo docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:dcba6daec718f547568c562956fa47e1b03673dd010fe6ee58ca806767031d1c
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https

In [13]:
sudo usermod -aG docker $USER

*NB*: restart jupyter from a fresh shell session to apply new GID.

## Log into docker registry

Get your self a docker registry (see common/Registry.ipynb).

In [ ]:
REPOSITORY=grepo.azurecr.io

```
# docker login $REPOSITORY --username *** --password ***
```

## Build patched moby

In [27]:
WD=$(mktemp -d)

### fetch source code (master branch)

In [ ]:
git clone https://github.com/abbbe/moby.git
cd moby

### install dependencies

In [ ]:
sudo snap install go --classic
go get -d ./...
go install github.com/LK4D4/vndr@latest

*NB*: vndr executable should end up in $PATH for instance in ~/.local/bin

### test unmodified code

In [ ]:
make binary

In [ ]:
make test

In [ ]:
TESTDIRS=./container/stream make test-unit

### patch and test

In [ ]:
git checkout black_is_white

In [ ]:
make clean
make binary

In [ ]:
TESTDIRS=./container/stream make test-unit

In [ ]:
TEST_SKIP_INTEGRATION_CLI=1 make test-integration TESTFLAGS="-test.run TestExecBlackIsWhite"

The last command takes a long time to run. Also, it runs tests from various directories and you do no necessarily see if your tests has failed.

```
...
Running /go/src/github.com/docker/docker/integration/container (amd64.integration.container) flags=-test.v -test.timeout=5m -test.run TestExecBlackIsWhite 
INFO: Testing against a local daemon
=== RUN   TestExecBlackIsWhite
--- PASS: TestExecBlackIsWhite (0.65s)
PASS

DONE 1 tests in 0.677s
...
```

To quickly run our test only:

```
$ make BIND_DIR=. shell
# ./bundles/dynbinary-daemon/dockerd &
# go test -v ./integration/container -run TestExecBlackIsWhite
Loaded image: busybox:latest
Loaded image: busybox:glibc
Loaded image: debian:bullseye-slim
Loaded image: hello-world:latest
Loaded image: arm32v7/hello-world:latest
INFO: Testing against a local daemon
=== RUN   TestExecBlackIsWhite
time="2023-08-27T18:21:36.197724895Z" level=info msg="loading plugin \"io.containerd.internal.v1.shutdown\"..." runtime=io.containerd.runc.v2 type=io.containerd.internal.v1
time="2023-08-27T18:21:36.197773274Z" level=info msg="loading plugin \"io.containerd.ttrpc.v1.pause\"..." runtime=io.containerd.runc.v2 type=io.containerd.ttrpc.v1
time="2023-08-27T18:21:36.197785044Z" level=info msg="loading plugin \"io.containerd.event.v1.publisher\"..." runtime=io.containerd.runc.v2 type=io.containerd.event.v1
time="2023-08-27T18:21:36.197792335Z" level=info msg="loading plugin \"io.containerd.ttrpc.v1.task\"..." runtime=io.containerd.runc.v2 type=io.containerd.ttrpc.v1
INFO[2023-08-27T18:21:36.498614412Z] shim disconnected                             id=a2852cf866651e8283a6808fe3058e4c1a35b2d26097a800877cbade99bebf5d namespace=moby
WARN[2023-08-27T18:21:36.498685085Z] cleaning up after shim disconnected           id=a2852cf866651e8283a6808fe3058e4c1a35b2d26097a800877cbade99bebf5d namespace=moby
INFO[2023-08-27T18:21:36.498752438Z] ignoring event                                container=a2852cf866651e8283a6808fe3058e4c1a35b2d26097a800877cbade99bebf5d module=libcontainerd namespace=moby topic=/tasks/delete type="*events.TaskDelete"
INFO[2023-08-27T18:21:36.498787843Z] cleaning up dead shim                         namespace=moby
--- PASS: TestExecBlackIsWhite (0.69s)
PASS
ok  	github.com/docker/docker/integration/container	1.919s
```

### leave moby/

In [4]:
cd ..

## Build docker:dind image with black-is-white feature

Supposedly we have to do bild in moby/, but it did not work for me. So we create an empty directory and build there.

#### build

In [28]:
cp moby/bundles/binary/dockerd $WD/

cat << END > $WD/Dockerfile.dind-black-is-white
# Use the official docker:dind as a base
FROM docker:dind

COPY ./dockerd /usr/local/bin/dockerd
END

In [29]:
docker build -t dind-black-is-white:latest -f $WD/Dockerfile.dind-black-is-white $WD

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (3/3)                                          docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile.dind-black-is-white   0.1s
 => => transferring dockerfile: 154B                                       0.0s
 => [internal] load metadata for docker.io/library/docker:dind             0.0s
[+] Building 0.2s (4/6)                                          docker:default
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile.dind-black-is-white   0.1s
 => => transferring dockerfile: 154B                                       0.0s
 => [internal] load metadata for docker.

In [30]:
docker kill dind-black-is-white
docker run --rm --name dind-black-is-white -d --privileged -it dind-black-is-white:latest

Error response from daemon: Cannot kill container: dind-black-is-white: No such container: dind-black-is-white
8e5df987057809dda786e101eae21f0368a71e6a4ee8d9b76be3f5661661699f


In [31]:
docker exec -it dind-black-is-white \
    docker --version

Docker version 24.0.5, build ced0996


In [33]:
docker exec -it dind-black-is-white \
    docker run busybox echo "{black}"

Unable to find image 'busybox:latest' locally
latest: Pulling from library/busybox

Digest: sha256:3fbc632167424a6d997e74f52b878d7cc478225cffac6bc977eedfe51c7f4e79
Status: Downloaded newer image for busybox:latest
{white}


Here we have dind-black-is-white image with black-is-white feature.

### push

In [37]:
docker tag dind-black-is-white $REPOSITORY/dind-black-is-white:latest

In [39]:
docker push $REPOSITORY/dind-black-is-white:latest

The push refers to repository [grepo.azurecr.io/dind-black-is-white]

0d0c0068: Preparing 
a8470c6e: Preparing 
79695dfb: Preparing 
5c9a229a: Preparing 
a81f3d2c: Preparing 
5f19aeb6: Preparing 
742599ab: Preparing 
443bfbbf: Preparing 
2878f7a3: Preparing 
f3153c61: Preparing 
b52abefd: Preparing 
8ef27873: Preparing 
bf18a086: Preparing 
6dc47777: Preparing 
latest: digest: sha256:25d3847dd853bda50eddd16f49749eb506cfd97fdd890e318003af8c09434e74 size: 3463


### P

In [42]:
cd moby

In [43]:
git status

On branch black_is_white
Your branch is up to date with 'origin/black_is_white'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   DinD-Black-Is-White.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
